<a href="https://colab.research.google.com/github/nguyenkimthach/Optimization_Binary_Neural_Network/blob/main/CNN_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install larq
!pip install tensorflow==2.7.2

In [ ]:
import tensorflow as tf
import larq as lq
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
num_classes = 10

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

train_images = train_images.reshape((50000, 32, 32, 3)).astype("float32")
test_images = test_images.reshape((10000, 32, 32, 3)).astype("float32")

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

In [ ]:
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D,Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from keras.models import Sequential

model = Sequential()

# filters 32, size of filters (3,3)
model.add(Conv2D(16, (3, 3), input_shape=(32, 32, 3), padding='same'))
model.add(BatchNormalization(axis=-1)) #BatchNormalization normalizes the matrix after it is been through a convolution layer so that the scale of each dimension remains the same
model.add(Activation('relu'))

model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) # act as an input to the Dense layers.

# Fully connected layer
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax')) # Softmax activation enables us to calculate the output based on the probabilities

## Model Training

Compile the model and train the model

In [ ]:
model.summary()
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
trained_model = model.fit(
    train_images,
    train_labels,
    batch_size=50,
    epochs=30,
    validation_data=(test_images, test_labels),
    shuffle=True
)

## Model Output

We can now plot the final validation accuracy and loss:

In [ ]:
paras = trained_model.history['accuracy']
paras.insert(0,0.25)
plt.plot(paras)
plt.plot(trained_model.history['accuracy'])
#plt.plot(trained_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

print(np.max(trained_model.history['accuracy']))

In [ ]:
paras = trained_model.history['loss']
paras.insert(0,1.6)
plt.plot(paras)
#plt.plot(trained_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

print(np.min(trained_model.history['loss']))
print(np.min(trained_model.history['val_loss']))